# IBM Cloud Pak for Data Data Virtualization Lab Data Engineer Exploration with REST

### Where to find this sample online
You can find a copy of this notebook at https://github.com/Db2-DTE-POC/db2dmc.

### First we will import a few helper classes
We need to pull in a few standard Python libraries so that we can work with REST, JSON and a library called Pandas. Pandas lets us work with DataFrames, which are a very powerful way to work with tabular data in Python. 

In [1]:
# Import the class libraries 
import requests
import ssl
import json
from pprint import pprint
from requests import Response
import pandas as pd
import time
from requests.packages.urllib3.exceptions import InsecureRequestWarning
requests.packages.urllib3.disable_warnings(InsecureRequestWarning)
from IPython.display import IFrame
from IPython.display import display, HTML
from pandas.io.json import json_normalize
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

### The Db2 Class
Next we will create a Db2 helper class that will encapsulate the Rest API calls that we can use to directly access the Db2 Data Management Console service without having to use the user interface. 

To access the service we need to first authenticate with the service and create a reusable token that we can use for each call to the service. This ensures that we don't have to provide a userID and password each time we run a command. The token makes sure this is secure. 

Each request is constructed of several parts. First, the URL and the API identify how to connect to the service. Second the REST service request that identifies the request and the options. For example '/metrics/applications/connections/current/list'. And finally some complex requests also include a JSON payload. For example running SQL includes a JSON object that identifies the script, statement delimiters, the maximum number of rows in the results set as well as what do if a statement fails.

The full set of APIs are documents as part of the Db2 Data Management Console user interface. In this hands on lab you can connect to that directly through this link: [Db2 Data Management Console RESTful APIs](http://localhost:11080/dbapi/api/index_enterprise.html). 

In [2]:
# Run the Db2 Class library
# Used to construct and reuse an Autentication Key
# Used to construct RESTAPI URLs and JSON payloads
class Db2():
    
    def __init__(self, url, verify = False, proxies=None, ):
        self.url = url
        self.proxies = proxies
        self.verify = verify

    def authenticate(self, api, userid, password):
        
        credentials = {'username':userid, 'password':password}
        r = requests.post(self.url+api+'/preauth/signin', verify=self.verify, json=credentials, proxies=self.proxies)
        if (r.status_code == 200):
            bearerToken = "Bearer " + r.cookies["ibm-private-cloud-session"]
            print(bearerToken)
            self.headers = {'Content-Type':"application/json", 'Accept':"application/json", 'Authorization': bearerToken, 'Cache-Control': "no-cache"}
        else:
            print ('Unable to authenticate, no bearer token obtained')
        
    def printResponse(self, r, code):
        if (r.status_code == code):
            pprint(r.json())
        else:
            print (r.status_code)
            print (r.content)
    
    def getRequest(self, api, json=None):
        return requests.get(self.url+api, verify = self.verify, headers=self.headers, proxies = self.proxies, json=json)

    def postRequest(self, api, json=None):
        return requests.post(self.url+api, verify = self.verify, headers=self.headers, proxies = self.proxies, json=json) 
    
    def deleteRequest(self, api, json=None):
        return requests.delete(self.url+api, verify = self.verify, headers=self.headers, proxies = self.proxies, json=json) 
        
    def getStatusCode(self, response):
        return (response.status_code)

    def getJSON(self, response):
        return (response.json())
    
    def getDataSources(self):
        return self.getRequest('/icp4data-databases/dv/icp4d-test/dvapiserver/v1/dv/datasource_nodes')
    
    def getSchemas(self):
        return self.getRequest('/icp4data-databases/dv/icp4d-test/dbapi/v4/schemas')
    
    def runSQL(self, script, limit=10, separator=';', stopOnError=False):
        sqlJob = {'commands': script, 'limit':limit, 'separator':separator, 'stop_on_error':str(stopOnError)}
        return self.postRequest('/icp4data-databases/dv/icp4d-test/dbapi/v4/sql_jobs',sqlJob)
        
    def getSQLJobResult(self, jobid):
        return self.getRequest('/icp4data-databases/dv/icp4d-test/dbapi/v4/sql_jobs/'+jobid)
       
    def getSearchViewList(self, searchtext, show_systems="false"):
        return self.getRequest('/icp4data-databases/dv/icp4d-test/dbapi/v4/admin/schemas/obj_type/view?search_name='+searchtext+'&show_systems='+str(show_systems)+'&rows_return=200');
    
    def getSearchTableList(self, searchtext):
        return self.getRequest('/icp4data-databases/dv/icp4d-test/dbapi/v4/admin/schemas/obj_type/table?search_name='+searchtext+'&show_systems=true&rows_return=100');
               
    def postSearchObjects(self, obj_type, search_text, rows_return=100, show_systems='false', is_ascend='true'):     
        json = {"search_name":search_text,"rows_return":rows_return,"show_systems":show_systems,"obj_type":obj_type,"filters_match":"ALL","filters":[]}       
        return self.postRequest('/icp4data-databases/dv/icp4d-test/dbapi/v4/admin/'+str(obj_type)+'s',json);
            
    def getTablesInSchema(self, schema):
        return self.getRequest('/icp4data-databases/dv/icp4d-test/dbapi/v4/schemas/'+str(schema)+'/tables'); 
    
    def getVirtualizedTables(self):
        return self.getRequest('/icp4data-databases/dv/icp4d-test/dvapiserver/v1/dv/mydata/tables')
    
    def getVirtualizedTablesDF(self):
        r = self.getVirtualizedTables()
        if (databaseAPI.getStatusCode(r)==200):
            json = databaseAPI.getJSON(r)
            df = pd.DataFrame(json_normalize(json['tables']))
            return df
        else:
            print(databaseAPI.getStatusCode(r))

    def getVirtualizedViews(self):
        return self.getRequest('/icp4data-databases/dv/icp4d-test/dvapiserver/v1/dv/mydata/views')
    
    def getVirtualizedViewsDF(self):
        r = self.getVirtualizedViews()
        if (databaseAPI.getStatusCode(r)==200):
            json = databaseAPI.getJSON(r)
            df = pd.DataFrame(json_normalize(json['views']))
            return df
        else:
            print(databaseAPI.getStatusCode(r))
    
    def grantPrivledgeToRole(self, objectName, objectSchema, roleToGrant):
        json =   {"objectName":objectName,"objectSchema":objectSchema,"roleToGrant":roleToGrant}
        return self.postRequest('/icp4data-databases/dv/icp4d-test/dvapiserver/v1/privileges/roles',json);
 
    def getRole(self, role):
        return self.getRequest('/icp4data-databases/dv/icp4d-test/dvapiserver/v1/privileges/objects/role/'+str(role));
    
    def foldData(self, sourceName, sourceTableDef, sources ):
        json = {"sourceName":sourceName,"sourceTableDef":sourceTableDef,"sources":sources}
        return self.postRequest('/icp4data-databases/dv/icp4d-test/dvapiserver/v1/dv/virtualize/tables', json);

    def addUser(self, username, displayName, email, user_roles, password):
        json = {"username":username,"displayName":displayName,"email":email,"user_roles":user_roles,"password":password}
        return self.postRequest('/api/v1/usermgmt/v1/user', json);
    
    def dropUser(self, username):
        return self.deleteRequest('/api/v1/usermgmt/v1/user/'+str(username));
   
    def getUsers(self):
        return self.getRequest('/api/v1/usermgmt/v1/usermgmt/users');
    
    def getUsersDF(self):
        r = self.getUsers()
        if (databaseAPI.getStatusCode(r)==200):
            json = databaseAPI.getJSON(r)
            df = pd.DataFrame(json_normalize(json))
            return df
        else:
            print(databaseAPI.getStatusCode(r));
    
    def addUserToDV(self, display_name, role, usersDF):
        userrow = (usersDF.loc[usersDF['displayName'] == display_name])
        uid = userrow['uid'].values[0]
        username = userrow['username'].values[0]
        
        json = {"users":[{"uid":uid,"username":username,"display_name":display_name,"role":role}],"serviceInstanceID":"1573915078292"}
        return self.postRequest('/zen-data/v2/serviceInstance/users', json);
    
    def dropUserFromDV(self, display_name, usersDF):
        userrow = (usersDF.loc[usersDF['displayName'] == display_name])
        uid = userrow['uid'].values[0]
        
        json = {"users":[uid],"serviceInstanceID":"1573915078292"}
        return self.deleteRequest('/zen-data/v2/serviceInstance/users', json);
    


In [3]:
def runSQL(sqlText):

    # Run the SQL Script and return the runID for later reference 
    runID = databaseAPI.getJSON(databaseAPI.runSQL(sqlText))['id'] 

    # See if there are any results yet for this job
    json = databaseAPI.getJSON(databaseAPI.getSQLJobResult(runID))

    # If the REST call returns an error return the json with the error to the calling routine
    if 'errors' in json :
        return json
    # Append the results from each statement in the script to the overall combined JSON result set
    fulljson = json

    while json['results'] != [] or (json['status'] != "completed" and json['status'] != "failed") :
        json = databaseAPI.getJSON(databaseAPI.getSQLJobResult(runID))

        # Get the results from each statement as they return and append the results to the full JSON 
        for results in json['results'] :
            fulljson['results'].append(results)
        # Wait 250 ms for more results
        time.sleep(0.25) 
    return fulljson

print('runSQL routine defined')

runSQL routine defined


In [4]:
def displayResults(json):

    for results in json['results']:
        print('Statement: '+str(results['index'])+': '+results['command'])
        print('Runtime ms: '+str(results['runtime_seconds']*1000))
        if 'error' in results : 
            print(results['error'])
        elif 'rows' in results :
            df = pd.DataFrame(results['rows'],columns=results['columns'])
            print(df)
        else :
            print('No errors. Row Affected: '+str(results['rows_affected']))
        print()
print('displayResults routine defined')

displayResults routine defined


## Establishing a Connection to the Console

### Example Connections
To connect to the Data Virtualization service you need to provide the URL, the service name (v1) and profile the console user name and password. For this lab we are assuming that the following values are used for the connection:
* Userid: admin
* Password: password

In [21]:
# Connect to the Db2 Data Management Console service

# From Outside the Cluster
Console  = 'https://services-uscentral.skytap.com:9152'
# From Inside the Cluster
# Console  = 'https://openshift-skytap-nfs-lb.ibm.com'
user     = 'labdataengineer1'
password = 'password'

# Set up the required connection
databaseAPI = Db2(Console)
api = '/v1'
databaseAPI.authenticate(api, user, password)
database = Console

Bearer eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCJ9.eyJ1c2VybmFtZSI6ImxhYmRhdGFlbmdpbmVlcjEiLCJzdWIiOiJsYWJkYXRhZW5naW5lZXIxIiwiaXNzIjoiS05PWFNTTyIsImF1ZCI6IkRTWCIsInJvbGUiOiJVc2VyIiwicGVybWlzc2lvbnMiOlsidmlydHVhbGl6ZV90cmFuc2Zvcm0iLCJtYW5hZ2VfaW5mb3JtYXRpb25fYXNzZXRzIiwiYWNjZXNzX3F1YWxpdHkiLCJtYW5hZ2VfZGlzY292ZXJ5IiwibWFuYWdlX21ldGFkYXRhX2ltcG9ydCIsImF1dGhvcl9nb3Zlcm5hbmNlX2FydGlmYWN0cyIsImFjY2Vzc19jYXRhbG9nIiwiY2FuX3Byb3Zpc2lvbiJdLCJ1aWQiOiIxMDAwMzMxMTMwIiwiYXV0aGVudGljYXRvciI6ImRlZmF1bHQiLCJkaXNwbGF5X25hbWUiOiJMQUJEQVRBRU5HSU5FRVIxIiwiaWF0IjoxNTg0NzQyNDMwLCJleHAiOjE1ODQ3ODU2MzB9.Zm50TnT-i5LdX1NTuzbywOXCNjw2K7UsK33DV0SERFpOLHFtuOYpnlBb9cYiLM1u_Q-7Oy4--eWqXq_x2oPaJP6zXaeU89vEhyudbjSr_vqADV882OHQQ44-jgmJ3lKklm5e790xY0JgXsoBgGhM__A44vQEC4Uc1xDfiFyXABXX1asAr61JMD39ES4dC2nidkO_mPz4AhVRb6pQZiOcqo2qwAIusfI_l2yuCesU1pKNSzt3szxF7BpUxZ7H-a3YYTIFOQcxA9e-C9e_xi6l54sa-EziR-t-yWpiFSvIhr-pVA7o-_4ftob6GE3E6OWLek_a7n2VeVTwyT6afQGZcA


## Explore Virtualization

In [22]:
r = databaseAPI.getDataSources()
if (databaseAPI.getStatusCode(r)==200):
    json = databaseAPI.getJSON(r)
    df = pd.DataFrame(json_normalize(json))
    print(', '.join(list(df)))
    display(df)
else:
    print(databaseAPI.getStatusCode(r))  

agent_class, dataSources, dscount, hostname, is_docker, node_description, node_name, os_user, port


,agent_class,dataSources,dscount,hostname,is_docker,node_description,node_name,os_user,port
0,H,None,0,dv-0,N,Not specified,AdminNode,bigsql,6414
1,H,"[{'cid': 'DB210200', 'dbname': 'azdb', 'srchos...",2,dv-0,Y,Not specified,qpendpoint_2:6416,bigsql,6416
2,H,"[{'cid': 'DVM10060', 'dbname': 'SQL92', 'srcho...",2,dv-0,Y,Not specified,qpendpoint_3:6417,bigsql,6417
3,H,"[{'cid': 'DB210113', 'dbname': 'BLUDB', 'srcho...",1,dv-0,Y,Not specified,qpendpoint_4:6418,bigsql,6418
4,H,"[{'cid': 'MONGO10213', 'dbname': 'mongo_onprem...",2,dv-0,Y,Not specified,qpendpoint_5:6419,bigsql,6419
5,H,"[{'cid': 'INFOR10146', 'dbname': 'STOCKS', 'sr...",1,dv-0,Y,Not specified,qpendpoint_1:6415,bigsql,6415


In [20]:
# Display the Virtualized Assets Avalable to Engineers and Users
roles = ['DV_ENGINEER','DV_USER']
for role in roles:
    r = databaseAPI.getRole(role)
    if (databaseAPI.getStatusCode(r)==200):
        json = databaseAPI.getJSON(r)
        df = pd.DataFrame(json_normalize(json['objects']))
        print(', '.join(list(df)))
        display(df)
    else:
        print(databaseAPI.getStatusCode(r))  

401
401


In [8]:
### Display Virtualized Tables and Views 
display(databaseAPI.getVirtualizedTablesDF())
display(databaseAPI.getVirtualizedViewsDF())

,create_time,data_source_table_name,owner,table_name,table_schema
0,2020-03-10T15:09:26.775819Z,STOCK_HISTORY,USER999,STOCK_HISTORY,FOLDING
1,2020-03-10T16:28:03.30376Z,ACCOUNTS,USER999,ACCOUNTS,FOLDING
2,2020-03-17T19:21:33.232495Z,CUSTOMER_PAYMENT,USER999,CUSTOMER_PAYMENT,MONGO_ONPREM
3,2020-03-19T13:14:31.00017Z,STOCK_TRANSACTIONS,USER999,STOCK_TRANSACTIONS_TEST,FOLDING
4,2020-03-17T19:21:32.954595Z,CUSTOMER_CONTACT,USER999,CUSTOMER_CONTACT,MONGO_ONPREM
5,2020-03-17T19:21:33.163397Z,CUSTOMER_IDENTITY,USER999,CUSTOMER_IDENTITY,MONGO_ONPREM
6,2020-03-12T15:31:05.763058Z,STOCK_SYMBOLS_VSAM,USER999,STOCK_SYMBOLS,DVDEMO
7,2020-03-11T13:21:58.342311Z,STOCK_TRANSACTIONS,USER999,STOCK_TRANSACTIONS,FOLDING


,create_time,owner,viewname,viewschema
0,2020-03-17T19:31:33.220694Z,USER999,CUSTOMERS,TRADING
1,2020-03-17T19:21:32.594375Z,USER999,CUSTOMER,MONGO_ONPREM
2,2020-03-11T16:29:32.48664Z,USER999,THREEPERCENT,TRADING
3,2020-03-11T16:29:32.714582Z,USER999,TRANSBYCUSTOMER,TRADING
4,2020-03-16T19:04:13.13363Z,USER999,CUSTOMERS,DVDEMO
5,2020-03-11T16:29:32.932202Z,USER999,TOPBOUGHTSOLD,TRADING
6,2020-03-16T19:35:35.534956Z,USER999,OHIO,TRADING
7,2020-03-11T16:29:33.156362Z,USER999,TOPFIVE,TRADING
8,2020-03-11T16:29:33.30725Z,USER999,BOTTOMFIVE,TRADING
9,2020-03-13T15:13:36.142598Z,USER999,THREESTOCKS,TRADING


## Manage Users and Access

### Cloud Pak for Data User Management

In [11]:
# Get the list of CPD Users
r = databaseAPI.getUsers()
if (databaseAPI.getStatusCode(r)==200):
    json = databaseAPI.getJSON(r)
    df = pd.DataFrame(json_normalize(json))
    print(', '.join(list(df)))
    display(df[['uid','username','displayName']])
else:
    print(databaseAPI.getStatusCode(r))

uid, deletable, username, displayName, authenticator, role, permissions, user_roles, email, default_user, created_timestamp, last_modified_timestamp, approval_status, current_account_status, internal_user


,uid,username,displayName
0,1000330999,admin,admin
1,1000331001,ctp,ctp
2,1000331004,dteuser,DTEUSER
3,1000331007,labadmin,LABADMIN
4,1000331008,labdataengineer,LABDATAENGINEER
5,1000331128,labdataengineer0,LABDATAENGINEER0
6,1000331130,labdataengineer1,LABDATAENGINEER1
7,1000331132,labdataengineer2,LABDATAENGINEER2
8,1000331134,labdataengineer3,LABDATAENGINEER3
9,1000331136,labdataengineer4,LABDATAENGINEER4


## Run SQL Against the DV Service

### Add and Remove Users to and from the DV Service

In [12]:
sqlText = \
'''
WITH MAX_VOLUME(AMOUNT) AS (
  SELECT MAX(VOLUME) FROM FOLDING.STOCK_HISTORY
    WHERE SYMBOL = 'DJIA'
),
HIGHDATE(TX_DATE) AS (
  SELECT TX_DATE FROM FOLDING.STOCK_HISTORY, MAX_VOLUME M
    WHERE SYMBOL = 'DJIA' AND VOLUME = M.AMOUNT
),
CUSTOMERS_IN_OHIO(CUSTID) AS (
  SELECT C.CUSTID FROM DVDEMO.CUSTOMERS C 
    WHERE C.STATE = 'OH'
),
TOTAL_BUY(CUSTID,TOTAL) AS (
  SELECT C.CUSTID, SUM(SH.QUANTITY * SH.PRICE) 
    FROM CUSTOMERS_IN_OHIO C, FOLDING.STOCK_TRANSACTIONS SH, HIGHDATE HD
  WHERE SH.CUSTID = C.CUSTID AND
        SH.TX_DATE = HD.TX_DATE AND 
        QUANTITY > 0 
  GROUP BY C.CUSTID
)
SELECT C.LASTNAME, T.TOTAL 
  FROM DVDEMO.CUSTOMERS C, TOTAL_BUY T
WHERE C.CUSTID = T.CUSTID
  ORDER BY TOTAL DESC
FETCH FIRST 5 ROWS ONLY;
'''

displayResults(runSQL(sqlText))

Statement: 0: WITH MAX_VOLUME(AMOUNT) AS (
  SELECT MAX(VOLUME) FROM FOLDING.STOCK_HISTORY
    WHERE SYMBOL = 'DJIA'
),
HIGHDATE(TX_DATE) AS (
  SELECT TX_DATE FROM FOLDING.STOCK_HISTORY, MAX_VOLUME M
    WHERE SYMBOL = 'DJIA' AND VOLUME = M.AMOUNT
),
CUSTOMERS_IN_OHIO(CUSTID) AS (
  SELECT C.CUSTID FROM DVDEMO.CUSTOMERS C 
    WHERE C.STATE = 'OH'
),
TOTAL_BUY(CUSTID,TOTAL) AS (
  SELECT C.CUSTID, SUM(SH.QUANTITY * SH.PRICE) 
    FROM CUSTOMERS_IN_OHIO C, FOLDING.STOCK_TRANSACTIONS SH, HIGHDATE HD
  WHERE SH.CUSTID = C.CUSTID AND
        SH.TX_DATE = HD.TX_DATE AND 
        QUANTITY > 0 
  GROUP BY C.CUSTID
)
SELECT C.LASTNAME, T.TOTAL 
  FROM DVDEMO.CUSTOMERS C, TOTAL_BUY T
WHERE C.CUSTID = T.CUSTID
  ORDER BY TOTAL DESC
FETCH FIRST 5 ROWS ONLY
Runtime ms: 342.99999475479126
    LASTNAME    TOTAL
0       Kirk  5358.66
1      Tyler  4876.95
2  Valentine  3350.08
3  Mccormick  3163.36
4    Vaughan  3018.40



In [13]:
repeat = 5
sqlText = 'SELECT * FROM TRADING.OHIO'

for x in range(0, repeat):
    print('Repetition number: '+str(x))
    runSQL(sqlText)
print('done')

Repetition number: 0
Repetition number: 1
Repetition number: 2
Repetition number: 3
Repetition number: 4
done


## Object Exploration

### List the Available Schemas in the Database

You can get the list of schemas through a REST service call. In this example the service call text was defined in the Db2 class at the start of the notebook. By default it includes both user and catalog schemas. 

If the call is successful it will return a 200 status code. The API call returns a JSON structure that we turn into a DataFrame using the normalize function. You can then list the columns of data available in the Data Frame and display the first 10 rows in the data frame. 

Many of the examples below list the columns available in the dataframe to make it easier for you to adapt the examples to your own needs. 

In [14]:
r = databaseAPI.getSchemas()
if (databaseAPI.getStatusCode(r)==200):
    json = databaseAPI.getJSON(r)
    df = pd.DataFrame(json_normalize(json['resources']))
    print(', '.join(list(df)))
    display(df[['name']].head(10))
else:
    print(databaseAPI.getStatusCode(r))   

definertype, name


,name
0,AWS
1,AWSMONGO
2,AZDB
3,AZMONGO
4,AZPOST
5,BIGSQL
6,CACHESYS
7,DB2DW-LOCAL
8,DB2DWHINT
9,DB2W0C


In [15]:
# Search for tables across all schemas that match simple search critera 
# Display the first 100
# Switch between searching tables or views
object = 'view'
# object = 'table'
r = databaseAPI.postSearchObjects(object,"TRADING",10,'false','false')
if (databaseAPI.getStatusCode(r)==200):
    json = databaseAPI.getJSON(r)
    df = pd.DataFrame(json_normalize(json))
    print('Columns:')
    print(', '.join(list(df)))
    display(df[[object+'_name']].head(100))
else:
    print("RC: "+str(databaseAPI.getStatusCode(r)))

Columns:
view_name, view_schema, owner, owner_type, read_only, valid, view_check, sql, create_time, alter_time, stats_time, optimize_query


,view_name
0,CUSTOMERS
1,THREESTOCKS
2,BOTTOMFIVE
3,TOPFIVE
4,TOPBOUGHTSOLD
5,TRANSBYCUSTOMER
6,THREEPERCENT
7,VOLUME
8,MOVING_AVERAGE
9,OHIO


In [17]:
sqlText = \
'''
SELECT * FROM TRADING.OHIO;
'''

displayResults(runSQL(sqlText))

Statement: 0: SELECT * FROM TRADING.OHIO
Runtime ms: 105.99999874830246
    LASTNAME    TOTAL
0      Boone  2098.25
1    Burgess  1565.16
2      Hicks  1362.56
3      Tyler  4876.95
4  Gallagher  2334.57
5    Navarro   704.13
6    Vaughan  3018.40
7  Gillespie  2002.50
8       Kirk  5358.66
9  Valentine  3350.08



### Next Steps
Try the [Analysing SQL Workloads](http://localhost:8888/notebooks/Db2_Data_Management_Console_SQL.ipynb). It contains extensive examples on how to run workloads that contain multiple SQL Statements across multiple databases and then measure their performance. 

Also try building some of your own reports based on the examples in this hands on lab. There are additional functions included in the Db2 class that we haven't explored yet in this lab. You can also include the Db2 class into your own notebook by including the [dmc_setup notebook](http://localhost:8888/notebooks/dmc_setup.ipynb)

#### Credits: IBM 2019, Peter Kohlmann [kohlmann@ca.ibm.com]